# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [2]:
import pandas as pd
import re
import numpy as np
import gensim
import nltk
from gensim.models.word2vec import Word2Vec
import string
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords


In [3]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [4]:
print(" ".join(whitespace_string.split()))
    

This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [5]:
with open('dates.txt', 'r', encoding='utf-8') as f:
  contents = f.read()
  
contents

'March 8, 2015\nMarch 15, 2015\nMarch 22, 2015\nMarch 29, 2015\nApril 5, 2015\nApril 12, 2015\nApril 19, 2015\nApril 26, 2015\nMay 3, 2015\nMay 10, 2015\nMay 17, 2015\nMay 24, 2015\nMay 31, 2015\nJune 7, 2015\nJune 14, 2015\nJune 21, 2015\nJune 28, 2015\nJuly 5, 2015\nJuly 12, 2015\nJuly 19, 2015'

In [6]:
regex = r"([a-zA-Z]+) (\d+), (\d\d\d\d)" 

search_result = re.findall(regex, contents)

for match in search_result:
  print(match)

('March', '8', '2015')
('March', '15', '2015')
('March', '22', '2015')
('March', '29', '2015')
('April', '5', '2015')
('April', '12', '2015')
('April', '19', '2015')
('April', '26', '2015')
('May', '3', '2015')
('May', '10', '2015')
('May', '17', '2015')
('May', '24', '2015')
('May', '31', '2015')
('June', '7', '2015')
('June', '14', '2015')
('June', '21', '2015')
('June', '28', '2015')
('July', '5', '2015')
('July', '12', '2015')
('July', '19', '2015')


In [7]:
dfDate = pd.DataFrame(search_result, columns=['Month', 'Day','Year'])
dfDate.head(100)

,Month,Day,Year
0,March,8,2015
1,March,15,2015
2,March,22,2015
3,March,29,2015
4,April,5,2015
5,April,12,2015
6,April,19,2015
7,April,26,2015
8,May,3,2015
9,May,10,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


#### Load data

In [8]:
#  
url='https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv'
df=pd.read_csv(url)
df.head()
df2=df.copy()

In [9]:
df['SentimentText'] = df['SentimentText'].str.lower()
df.head()
df2=df.copy()

In [10]:
stop_words = stopwords.words('english')
table = str.maketrans('','', string.punctuation)
stop_words = set(stopwords.words('english'))

In [11]:
df.head()

,Sentiment,SentimentText
0,0,is so sad for my apl frie...
1,0,i missed the new moon trail...
2,1,omg its already 7:30 :o
3,0,.. omgaga. im sooo im gunna cry. i'...
4,0,i think mi bf is cheating on me!!! ...


In [12]:
df=df2.copy()
def clean_doc(doc):
	# split into tokens by white space
	#tokens = str(doc).split(' ')
    tokens = word_tokenize(doc)
    #  lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    # remove punctuation from each token
    no_punctuation = [w.translate(table) for w in lowercase_tokens]
    # Remove words that aren't alphabetic
    alphabetic = [word for word in no_punctuation if word.isalpha()]
    # filter out stop words
    words = [w for w in alphabetic if not w in stop_words]
	# filter out short tokens
	#tokens = [word for word in words if len(word)>1]
    return words

df['SentimentText'] = df['SentimentText'].apply(clean_doc)
df2=df.copy()

In [13]:
df['SentimentText'] = df.SentimentText.apply(' '.join)

In [14]:
df.head()

,Sentiment,SentimentText
0,0,sad apl friend
1,0,missed new moon trailer
2,1,omg already
3,0,omgaga im sooo im gunna cry dentist since supo...
4,0,think mi bf cheating tt


### How should TF-IDF scores be interpreted? How are they calculated?

#### Term Frequency / Inverse Document Frequency  #####

Term Frequency: Percentage of words in document for each word
  TF = num of times word occurs (frequency) divided by the total num of word in   the document

Inverse Document Frequency: A penalty for the word existing in a high number of documents.
    IDF = log2(total-#-documents/ #-documents-including-word )

TF-IDF weighs a keyword in any content and assigns the importance to that keyword based on the number of times it appears in the document. It also checks how relevant the keyword is. Each word has its respective TF and IDF score. The product of the TF and IDF scores of a term is called the TF-IDF weight of that term.

Put simply, the higher the TF-IDF score (weight), the rarer the term and vice versa.


# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [15]:
from sklearn.model_selection import train_test_split

X = df.SentimentText.tolist()
y = df.Sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=9000, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'ca': 1929, 'nt': 5783, 'sleep': 7419, 'suggest': 7840, 'watching': 8645, 'quot': 6578, 'girlfriend': 3940, 'experience': 3497, 'ii': 4512, 'school': 7096, 'haha': 4140, 'reminding': 6757, 'case': 2083, 'forgot': 3763, 'la': 4929, 'love': 5193, 'country': 2731, 'music': 5597, 'abeeliever': 29, 'little': 5114, 'cake': 1947, 'got': 4018, 'ta': 7941, 'dont': 3176, 'know': 4901, 'coffee': 2529, 'changes': 2200, 'colour': 2555, 'lt': 5209, 'gt': 4099, 'everybody': 3457, 'dancing': 2903, 'pop': 6332, 'twitpic': 8359, 'working': 8823, 'photo': 6177, 'alas': 318, 'wan': 8617, 'na': 5606, 'hear': 4248, 'brian': 1728, 'bet': 1368, 'cute': 2869, 'like': 5075, 'voice': 8577, 'friend': 3809, 'told': 8179, 'youtube': 8963, 'video': 8546, 'tay': 7988, 'download': 3195, 'testing': 8053, 'finished': 3667, 'maaan': 5227, 'noticed': 5771, 'replied': 6777, 'week': 8676, 'ago': 235, 'edit': 3326, 'rooms': 6938, 'speed': 7588, 'http': 4437, 'time': 8144, 'long': 5157, 'wikipedia': 8739, 'failed': 3537, 'gu

In [24]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(79991, 9000)


,aa,aaa,aaah,aaahh,aafreen,aah,aahhh,aalaap,aamyfish,aaron,...,zealand,zero,zip,zombie,zombies,zone,zoo,zoom,zune,ðµ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
 #vectorize  X_test, uses ame vocabulary as the training dataset so  just call .transform() on X_test
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(19998, 9000)


,aa,aaa,aaah,aaahh,aafreen,aah,aahhh,aalaap,aamyfish,aaron,...,zealand,zero,zip,zombie,zombies,zone,zoo,zoom,zune,ðµ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

C:\Users\EDC\Anaconda3\envs\environ1\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.9659461689440062
Test Accuracy: 0.711021102110211


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [27]:
from gensim.models import Word2Vec
w2v = Word2Vec(df2.SentimentText, min_count=20, window=3, size=300, negative=20)

In [31]:
w2v.wv.most_similar('tweet', topn=10)

[('message', 0.7595947980880737),
 ('included', 0.7340092658996582),
 ('reply', 0.7211431264877319),
 ('list', 0.7027485370635986),
 ('follow', 0.7008450031280518),
 ('longest', 0.6843441128730774),
 ('sent', 0.6731915473937988),
 ('tweets', 0.6729306578636169),
 ('direct', 0.6724131107330322),
 ('comment', 0.6710553765296936)]